In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import re
from collections import defaultdict

import torchtext
from torchtext.legacy.data import Example

from wikipedia2vec import Wikipedia2Vec

In [2]:
# Data with Entities

ROOT_FOLDER = "/raid6/home/ramraj/2021/ir/entity-reranking/Entity-Linking/"

# BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "benchmark-train-relevance-v2.0")
# BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "test-data", "ramraj-test-data-top100-BM25.json")
BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "Train-with-entities")
BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "Test-with-entities/ramraj-test-data-top100-BM25-opt.json")

# BENCHMARK_TRAIN_FOLD_FOLDER = os.path.join(ROOT_FOLDER, "Train-with-entities-sepEntAndWords")
# BENCHMARK_TEST_FILE = os.path.join(ROOT_FOLDER, "Test-with-entities/ramraj-test-data-top100-BM25-opt-entWordSEP.json")

# laod train

In [3]:
train_files = sorted(glob.glob(os.path.join(BENCHMARK_TRAIN_FOLD_FOLDER, "fold-*.json")))
print( len(train_files) )

train_data = []
for train_file in train_files:
    tmp_data = json.load(open(train_file, 'r'))
    train_data.extend(tmp_data)
    
len(train_data)

5


1937

# load test

In [4]:
test_data = json.load(open(BENCHMARK_TEST_FILE, 'r'))
len(test_data)

2254

# utils

In [6]:
# def tokenise(text):
#     # Replace annoying unicode with a space
#     text = re.sub(r'[^\x00-\x7F]+', ' ', text)
#     # The following replacements are suggested in the paper
#     # BidAF (Seo et al., 2016)
#     text = text.replace("''", '" ')
#     text = text.replace("``", '" ')

#     # Space out punctuation
#     space_list = "!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~"
#     space_list = "!\"#$%&()*+,./:;<=>?@[\\]^_`{|}~"
#     text = text.translate(str.maketrans({key: " {0} ".format(key) for key in space_list}))

#     # space out singlequotes a bit better (and like stanford)
#     text = text.replace("'", " '")
    
#     # use any APIs
#     text = text.replace('\t', '').replace('\n', '').lower().strip()
#     return text

In [5]:
# def alpha_numeric_filtering(text):
#     regex = re.compile('[^a-zA-Z]')
#     #First parameter is the replacement, second parameter is your input string
#     return regex.sub(' ', text)     

# load wikie

In [5]:
import torchtext.vocab as vocab

MODEL_FILE = "../entity-embedding-modelling/data/enwiki_20180420_300d.txt"
# https://github.com/pytorch/text/issues/722

custom_embeddings = vocab.Vectors(name = MODEL_FILE, cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")

In [9]:
# PKL_MODEL_FILE = "../entity-embedding-modelling/data/enwiki_20180420_100d.pkl"
# wiki2vec = Wikipedia2Vec.load(PKL_MODEL_FILE)

# vocab

In [6]:
PAD_TOKEN = '<PAD>'
SOS_TOKEN = '<SOS>'
UNK_TOKEN = '<UNK>'
EOS_TOKEN = '<EOS>'

MAX_Q_LENGTH = 20
MAX_D_LEN = 200

Q_TEXT = torchtext.legacy.data.Field(sequential=True, 
                                tokenize=lambda x: x.split('<SEP>'),
                                eos_token=EOS_TOKEN,
                                pad_token=PAD_TOKEN,
                                include_lengths=True,
                                batch_first=True,
                                fix_length=MAX_Q_LENGTH,
                                lower=True)

DOC_TEXT = torchtext.legacy.data.Field(sequential=True, 
                                tokenize=lambda x: x.split('<SEP>'),
                                eos_token=EOS_TOKEN,
                                pad_token=PAD_TOKEN,
                                include_lengths=True,
                                batch_first=True,
                                fix_length=MAX_D_LEN,
                                lower=True)

In [7]:
example_texts = []

FIELDS = [('doc_text', DOC_TEXT), ('q_text', Q_TEXT)]
# https://github.com/pytorch/text/issues/722


for train_data_sample in train_data:
    # q_text = train_data_sample['qString']
    q_text = '<SEP>'.join( [ e['entity_title'] for e in train_data_sample['qEntities'] ] )
    
    for rel_docs in train_data_sample['RelevantDocuments']:
        # doc_text = rel_docs['docText']
        # dEntities = [ e['entity_title'] for e in rel_docs['dEntities'] ]
        doc_text = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )

        
        example_texts.append( Example.fromlist([doc_text, q_text] , FIELDS ) )
        
for test_data_sample in test_data:
    # q_text = test_data_sample['qString']
    q_text = '<SEP>'.join( [ e['entity_title'] for e in test_data_sample['qEntities'] ] )
    
    for rel_docs in test_data_sample['RelevantDocuments']:
        # doc_text = rel_docs['docText']
        doc_text = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
        
        example_texts.append( Example.fromlist([doc_text, q_text] , FIELDS ) )

In [8]:
torchtext_dataset = torchtext.legacy.data.Dataset(example_texts, fields=FIELDS)

Q_TEXT.build_vocab(torchtext_dataset, vectors=custom_embeddings, vectors_cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")
DOC_TEXT.build_vocab(torchtext_dataset, vectors=custom_embeddings, vectors_cache="/raid6/home/ramraj/2021/ir/entity-reranking/entity-embedding-modelling/data/cache-enwiki")

print("======= Query ========")
print(len(Q_TEXT.vocab) )
print(Q_TEXT.vocab.vectors.size() )
print("======= Doc ========")
print( len(DOC_TEXT.vocab))
print( DOC_TEXT.vocab.vectors.size())

======= Query ========
2943
torch.Size([2943, 300])
======= Doc ========
171341
torch.Size([171341, 300])


In [9]:
print( DOC_TEXT.vocab.stoi['university of southampton'], DOC_TEXT.vocab.itos[14] )
print( DOC_TEXT.vocab.stoi['fahrenheit'], DOC_TEXT.vocab.itos[3] )

11557 disease
9974 human trafficking


# dataset

In [11]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

from pytorch_transformers import WarmupLinearSchedule, AdamW
from transformers import get_linear_schedule_with_warmup


def map_to_torch_float(encoding):
    encoding = torch.FloatTensor(encoding)
    encoding.requires_grad_(False)
    return encoding

def map_to_torch(encoding):
    encoding = torch.LongTensor(encoding)
    encoding.requires_grad_(False)
    return encoding


class TrainTRECDataset(Dataset):
    def __init__(self, raw_data, q_field, d_field, query_max_len=20, max_len=512, is_train=True):
        self.is_train = is_train
        self.max_len = max_len
        self.query_max_len = query_max_len
        
        self.q_field = q_field
        self.d_field = d_field

        self.qLookup = defaultdict()
        self.dLookup = defaultdict()
        self.data_df = self.load_data(raw_data)
        self.len = len(self.data_df)

    def load_data(self, data):
        return_data = {'qID': [], 'qText': [], 'dID': [], 'dText': [], 'label': []}
        for data_sample in data:
            qID = data_sample['qID']
            # qText = data_sample['qString']
            qText = '<SEP>'.join( [ e['entity_title'] for e in data_sample['qEntities'] ] )

            if qText == '': continue
            self.qLookup[qID] = qText

            for rel_docs in data_sample['RelevantDocuments']:
                dID = rel_docs['docID']
                # docText = rel_docs['docText']
                docText = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
                
                if docText == '': continue
                self.dLookup[dID] = docText

                return_data['qID'].append(qID)
                return_data['dID'].append(dID)
                # return_data['qText'].append( qText )
                # return_data['dText'].append( docText )

        # df = pd.DataFrame(return_data, columns=["qID", "qText", "dID", "dText"])
        df = pd.DataFrame(return_data, columns=["qID", "dID"])
        return df

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        index = index % self.len
        inst = self.data_df.loc[index]
        docID = inst['dID']
        qID = inst['qID']
        docText = self.dLookup[docID] # self.dLookup[docID] # inst['dText']
        qText = self.qLookup[qID] # self.qLookup[qID] # inst['qText']

        negInst = self.data_df[self.data_df['qID'] != qID].sample(n=3, replace=True, random_state=1).iloc[0]
        # negInst = self.data_df[self.data_df['qID'] != qID].iloc[0] # get a random element
        negDocID = negInst['dID']
        negDocText = self.dLookup[negDocID] # self.dLookup[negDocID] # negInst['dText']

        # =============== prepare text before feed ===============
        q_text_processed = self.q_field.numericalize( self.q_field.pad([ [l for l in self.q_field.preprocess(qText) if l.strip()] ]) )
        pos_d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(docText) if l.strip()] ]) )
        neg_d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(negDocText) if l.strip()] ]) )
        
        label = 1
        return q_text_processed, pos_d_text_processed, neg_d_text_processed, map_to_torch_float([label])


train_trec_dataset = TrainTRECDataset(train_data, Q_TEXT, DOC_TEXT)
print( train_trec_dataset.__len__() )

q, pos_d, neg_d, l = train_trec_dataset.__getitem__(1)

print(q[0].shape)
print(pos_d[0].shape)
print(neg_d[0].shape)
# print(q[0][0].shape)
# print(out[1][0].shape)
# print(out[2][0].shape)
# print()
# print(out[0][0].shape)

4699
torch.Size([1, 20])
torch.Size([1, 200])
torch.Size([1, 200])


In [12]:
q_str = []
for qq in q[0][0]:
    q_str.append(Q_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in pos_d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in neg_d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

fight-or-flight response urination evolutionary psychology evolutionary psychology point of view (philosophy) <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

evolutionary psychology stimulus (physiology) psychological abuse physical abuse fight-or-flight response biodiversity life <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

In [17]:
import random
from bunch import Bunch
import time
import datetime



CONFIG_FILE = "config.json"

def get_config_from_json(json_file):
    """
        Get the config from a json file
        :param json_file:
        :return: config(namespace) or config(dictionary)
        """
    # parse the configurations from the config json file provided
    with open(json_file, 'r') as config_file:
        config_dict = json.load(config_file)

    # convert the dictionary to a namespace using bunch lib
    config = Bunch(config_dict)

    return config, config_dict

import torch
config, _ = get_config_from_json(CONFIG_FILE)



config.data['results_dir'] = 'exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly'




seed_val = config.cmd_args['seed']
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
os.makedirs(config.data['results_dir'], exist_ok=True)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


train_dataloader = DataLoader(dataset=train_trec_dataset,
                              batch_size=config.training["train_batch_size"],
                              pin_memory=device,
                              num_workers=config.training['num_workers'],
                              shuffle=True)
n_train_batches = len(train_dataloader)
print("Number of train batches : ", n_train_batches)

cuda
Number of train batches :  294


In [19]:
for batch_data in train_dataloader:
    q, pos_d, neg_d, l = batch_data
    print(torch.squeeze(q[0]).shape)
    print(pos_d[0].shape)
    print(neg_d[0].shape)
    break

torch.Size([16, 20])
torch.Size([16, 1, 200])
torch.Size([16, 1, 200])


# Model

In [31]:
import typing
import torch.nn as nn
import torch.nn.functional as F


class SimScorer(nn.Module):
    def __init__(self, q_text_field, d_text_field, with_multi_layer_perceptron=True, mlp_num_layers=2, mlp_num_units=300, mlp_num_fan_out=1, padding_idx=1, embedding_freeze=True):
        super(SimScorer, self).__init__()
        self.with_multi_layer_perceptron = with_multi_layer_perceptron

        self.mlp_num_layers = mlp_num_layers
        self.mlp_num_units = mlp_num_units
        self.mlp_num_fan_out = mlp_num_fan_out
        self.q_text_field = q_text_field
        self.d_text_field = d_text_field
        self.padding_idx = padding_idx
        self.embedding_freeze = embedding_freeze
        self.build()
            
    def build(self):
        """
        Build model structure.
        """
        self.q_embedding = nn.Embedding.from_pretrained(self.q_text_field.vocab.vectors, padding_idx=self.padding_idx, freeze=self.embedding_freeze)
        self.d_embedding = nn.Embedding.from_pretrained(self.d_text_field.vocab.vectors, padding_idx=self.padding_idx, freeze=self.embedding_freeze)
        
        self.q_fc1 = self._make_perceptron_layer(300, 300)
        self.d_fc1 = self._make_perceptron_layer(300, 300)
        
    def _make_perceptron_layer(self, in_features=1, out_features=1, activation=nn.ReLU()):
        return nn.Sequential(nn.Linear(in_features, out_features), activation)

    def forward(self, inputs):
        input_left, input_right = inputs['text_left'], inputs['text_right']
        
        input_left_emb = self.q_embedding(input_left)
        input_right_emb = self.d_embedding(input_right)
        
#         print("input_left_emb : ", input_left_emb.shape)
#         print("input_right_emb : ", input_right_emb.shape)
        
        input_left_emb = self.q_fc1(input_left_emb)
        input_right_emb = self.d_fc1(input_right_emb)
        
#         print("input_left_emb : ", input_left_emb.shape)
#         print("input_right_emb : ", input_right_emb.shape)
        
        input_left_emb_avg = torch.mean(input_left_emb, dim=1)
        input_right_emb_avg = torch.mean(input_right_emb, dim=1)
        
        # print("input_left_emb_avg : ", input_left_emb_avg.shape)
        # print("input_right_emb_avg : ", input_right_emb_avg.shape)
    
        # Dot product with cosine similarity.
        x = F.cosine_similarity(input_left_emb_avg, input_right_emb_avg)
        # x = x.unsqueeze(dim=1)
        # print("x : ", x.shape)

        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

net = SimScorer(Q_TEXT, DOC_TEXT)
net.to(device)
print('Models built and ready to go!')

# tmp_q_text = torch.randint(low=1, high=30, size=(16, 20)).to(device)
# tmp_d_text = torch.randint(low=1, high=30, size=(16, 200)).to(device)
# print(tmp_q_text.shape)
# print(tmp_d_text.shape)

# input_text = {'text_left': tmp_q_text, 'text_right': tmp_d_text}

# output = net.forward(input_text)
# print(output)
# output.shape

cuda
Models built and ready to go!


In [32]:
LEARNING_RATE = 0.001 # 0.001 # 2e-5

criterion = nn.MarginRankingLoss(margin=1, size_average=True)
# criterion = nn.MarginRankingLoss(size_average=True)
opti = AdamW(net.parameters(),
             lr=LEARNING_RATE,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
             eps=1e-8,  # args.adam_epsilon  - default is 1e-8.
             correct_bias=False
             )

num_epochs = config.training['epochs']
display_step = config['training']['display_step']
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(opti,
                                            num_warmup_steps=0,  # Default value in run_glue.py
                                            num_training_steps=total_steps)

def format_time(elapsed_time):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed_time)))
    return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss

In [45]:
training_stats = []
history = defaultdict(list)

resume_epoch = 0
print_every = 1 # 10
save_every = 200 # 1000

print('Initializing ...')
print("Training...")


resume_epoch = 11
num_epochs = 20

total_t0 = time.time()
for epoch_idx in range(resume_epoch, num_epochs):
    total_loss = 0
    total_score = 0


    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_idx + 1, num_epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    net.train()  # TODO: IMPORTANT !
    for batch_idx, train_batch_data in enumerate(train_dataloader):
        # Clear gradients
        net.zero_grad()  # TODO: check validity !
        opti.zero_grad()
        
        # Converting these to cuda tensors
        q_text, pos_d_text, neg_d_text, label = train_batch_data

        q_text, pos_d_text, neg_d_text, label = torch.squeeze(q_text[0]).to(device), torch.squeeze(pos_d_text[0]).to(device), torch.squeeze(neg_d_text[0]).to(device), label.to(device)

        pos_net_output = net({'text_left': q_text, 'text_right': pos_d_text})
        neg_net_output = net({'text_left': q_text, 'text_right': neg_d_text})
        # # TODO: do i need a softmax or not ?

        # Computing loss
        loss = criterion(pos_net_output, neg_net_output, label.float())
        batch_loss = loss.item()
        total_loss += batch_loss

        # Back propagating the gradients
        loss.backward()
        if config.training['gradient_clipping']['use']:
            torch.nn.utils.clip_grad_norm_(net.parameters(), config.training['gradient_clipping']['clip_value'])

        # Optimization step
        opti.step()        

        with open(os.path.join(config.data['results_dir'], 'train-log-epoch.txt' ), 'a') as f:
                f.write(str(batch_idx+1) + '\t' + str(batch_idx+1) + '\t' + str(batch_loss) + '\t' + '\n')

        training_stats.append(
                {
                    'epoch': epoch_idx + 1,
                    'batch': batch_idx + 1,
                    'step': (epoch_idx * n_train_batches) + batch_idx + 1,
                    'Training Loss': batch_loss,
                    # 'Training Loss': avg_train_loss,
                    # 'Training Time': training_time,
                }
            )
                
        if batch_idx % print_every == 0: # Print progress
            total_loss_avg = total_loss / print_every 
            elapsed = format_time(time.time() - t0)
            print('| TRAIN SET | Epoch [{:02d}/{:02d}], Step [{:04d}/{:04d}], Loss: {:.4f} | Elapsed: {:}'
                          .format(epoch_idx+1, num_epochs, batch_idx, int(n_train_batches), total_loss_avg, elapsed))
            total_loss = 0              
        
        if ( (batch_idx == n_train_batches-1) or ((batch_idx+1) % save_every == 0) ): # Save checkpoint
            directory = os.path.join(config.data['results_dir'], 'ranking-pytorch-model')
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save(net.state_dict(),
                       os.path.join(directory, 'epoch-{}.batch-{}.{}.pt'.format(epoch_idx+1, batch_idx+1, 'checkpoint')))


    scheduler.step()  # TODO: IMPORTANT !

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

history_df = pd.DataFrame({"step": [e['step'] for e in training_stats],
                           "Training Loss": [e['Training Loss'] for e in training_stats]})
history_df.to_csv(os.path.join(config.data['results_dir'], "history.csv"), index=False)

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))

if not config.cmd_args['mode'] == "experiment":
    torch.save(net, os.path.join(config.data['results_dir'], "model-dump.pkl"))

Initializing ...
Training...

======== Epoch 12 / 20 ========
Training...
| TRAIN SET | Epoch [12/20], Step [0000/0294], Loss: 0.0594 | Elapsed: 0:00:00
| TRAIN SET | Epoch [12/20], Step [0001/0294], Loss: 0.0021 | Elapsed: 0:00:00
| TRAIN SET | Epoch [12/20], Step [0002/0294], Loss: 0.0009 | Elapsed: 0:00:00
| TRAIN SET | Epoch [12/20], Step [0003/0294], Loss: 0.0022 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0004/0294], Loss: 0.0034 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0005/0294], Loss: 0.0023 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0006/0294], Loss: 0.0016 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0007/0294], Loss: 0.0643 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0008/0294], Loss: 0.0012 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0009/0294], Loss: 0.0011 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0010/0294], Loss: 0.0050 | Elapsed: 0:00:01
| TRAIN SET | Epoch [12/20], Step [0011/0294], Loss: 0.00

| TRAIN SET | Epoch [12/20], Step [0104/0294], Loss: 0.0706 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0105/0294], Loss: 0.0014 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0106/0294], Loss: 0.0015 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0107/0294], Loss: 0.1200 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0108/0294], Loss: 0.1242 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0109/0294], Loss: 0.0010 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0110/0294], Loss: 0.0622 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0111/0294], Loss: 0.0650 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0112/0294], Loss: 0.0007 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0113/0294], Loss: 0.0010 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0114/0294], Loss: 0.1775 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0115/0294], Loss: 0.0644 | Elapsed: 0:00:06
| TRAIN SET | Epoch [12/20], Step [0116/0294], Loss:

| TRAIN SET | Epoch [12/20], Step [0210/0294], Loss: 0.0022 | Elapsed: 0:00:11
| TRAIN SET | Epoch [12/20], Step [0211/0294], Loss: 0.0014 | Elapsed: 0:00:11
| TRAIN SET | Epoch [12/20], Step [0212/0294], Loss: 0.0416 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0213/0294], Loss: 0.0862 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0214/0294], Loss: 0.0011 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0215/0294], Loss: 0.0016 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0216/0294], Loss: 0.0014 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0217/0294], Loss: 0.0019 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0218/0294], Loss: 0.0011 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0219/0294], Loss: 0.1021 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0220/0294], Loss: 0.0056 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0221/0294], Loss: 0.1257 | Elapsed: 0:00:12
| TRAIN SET | Epoch [12/20], Step [0222/0294], Loss:

| TRAIN SET | Epoch [13/20], Step [0022/0294], Loss: 0.0012 | Elapsed: 0:00:01
| TRAIN SET | Epoch [13/20], Step [0023/0294], Loss: 0.0666 | Elapsed: 0:00:01
| TRAIN SET | Epoch [13/20], Step [0024/0294], Loss: 0.0010 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0025/0294], Loss: 0.0042 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0026/0294], Loss: 0.0658 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0027/0294], Loss: 0.0650 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0028/0294], Loss: 0.0643 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0029/0294], Loss: 0.0629 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0030/0294], Loss: 0.0659 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0031/0294], Loss: 0.0015 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0032/0294], Loss: 0.0656 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0033/0294], Loss: 0.0636 | Elapsed: 0:00:02
| TRAIN SET | Epoch [13/20], Step [0034/0294], Loss:

| TRAIN SET | Epoch [13/20], Step [0131/0294], Loss: 0.0651 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0132/0294], Loss: 0.0662 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0133/0294], Loss: 0.0645 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0134/0294], Loss: 0.0346 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0135/0294], Loss: 0.0019 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0136/0294], Loss: 0.0015 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0137/0294], Loss: 0.0016 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0138/0294], Loss: 0.0042 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0139/0294], Loss: 0.0650 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0140/0294], Loss: 0.0013 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0141/0294], Loss: 0.0656 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0142/0294], Loss: 0.0882 | Elapsed: 0:00:06
| TRAIN SET | Epoch [13/20], Step [0143/0294], Loss:

...

| TRAIN SET | Epoch [19/20], Step [0154/0294], Loss: 0.0017 | Elapsed: 0:00:07
| TRAIN SET | Epoch [19/20], Step [0155/0294], Loss: 0.0677 | Elapsed: 0:00:07
| TRAIN SET | Epoch [19/20], Step [0156/0294], Loss: 0.0039 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0157/0294], Loss: 0.0040 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0158/0294], Loss: 0.0056 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0159/0294], Loss: 0.0643 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0160/0294], Loss: 0.0205 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0161/0294], Loss: 0.0310 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0162/0294], Loss: 0.0042 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0163/0294], Loss: 0.0239 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0164/0294], Loss: 0.0019 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0165/0294], Loss: 0.0009 | Elapsed: 0:00:08
| TRAIN SET | Epoch [19/20], Step [0166/0294], Loss:

| TRAIN SET | Epoch [19/20], Step [0261/0294], Loss: 0.0175 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0262/0294], Loss: 0.0634 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0263/0294], Loss: 0.0958 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0264/0294], Loss: 0.0017 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0265/0294], Loss: 0.0639 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0266/0294], Loss: 0.0020 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0267/0294], Loss: 0.0650 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0268/0294], Loss: 0.0035 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0269/0294], Loss: 0.0014 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0270/0294], Loss: 0.0174 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0271/0294], Loss: 0.0014 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0272/0294], Loss: 0.0025 | Elapsed: 0:00:13
| TRAIN SET | Epoch [19/20], Step [0273/0294], Loss:

| TRAIN SET | Epoch [20/20], Step [0071/0294], Loss: 0.0672 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0072/0294], Loss: 0.0143 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0073/0294], Loss: 0.0646 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0074/0294], Loss: 0.0060 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0075/0294], Loss: 0.0679 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0076/0294], Loss: 0.0057 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0077/0294], Loss: 0.0196 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0078/0294], Loss: 0.0656 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0079/0294], Loss: 0.0635 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0080/0294], Loss: 0.0181 | Elapsed: 0:00:04
| TRAIN SET | Epoch [20/20], Step [0081/0294], Loss: 0.1272 | Elapsed: 0:00:05
| TRAIN SET | Epoch [20/20], Step [0082/0294], Loss: 0.0019 | Elapsed: 0:00:05
| TRAIN SET | Epoch [20/20], Step [0083/0294], Loss:

| TRAIN SET | Epoch [20/20], Step [0176/0294], Loss: 0.0026 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0177/0294], Loss: 0.0039 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0178/0294], Loss: 0.0034 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0179/0294], Loss: 0.0028 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0180/0294], Loss: 0.0012 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0181/0294], Loss: 0.0016 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0182/0294], Loss: 0.0017 | Elapsed: 0:00:09
| TRAIN SET | Epoch [20/20], Step [0183/0294], Loss: 0.0029 | Elapsed: 0:00:10
| TRAIN SET | Epoch [20/20], Step [0184/0294], Loss: 0.0031 | Elapsed: 0:00:10
| TRAIN SET | Epoch [20/20], Step [0185/0294], Loss: 0.0035 | Elapsed: 0:00:10
| TRAIN SET | Epoch [20/20], Step [0186/0294], Loss: 0.0012 | Elapsed: 0:00:10
| TRAIN SET | Epoch [20/20], Step [0187/0294], Loss: 0.0103 | Elapsed: 0:00:10
| TRAIN SET | Epoch [20/20], Step [0188/0294], Loss:

| TRAIN SET | Epoch [20/20], Step [0281/0294], Loss: 0.0646 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0282/0294], Loss: 0.0203 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0283/0294], Loss: 0.0588 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0284/0294], Loss: 0.0692 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0285/0294], Loss: 0.0022 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0286/0294], Loss: 0.1264 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0287/0294], Loss: 0.0638 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0288/0294], Loss: 0.0691 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0289/0294], Loss: 0.0650 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0290/0294], Loss: 0.0099 | Elapsed: 0:00:15
| TRAIN SET | Epoch [20/20], Step [0291/0294], Loss: 0.0014 | Elapsed: 0:00:16
| TRAIN SET | Epoch [20/20], Step [0292/0294], Loss: 0.0028 | Elapsed: 0:00:16
| TRAIN SET | Epoch [20/20], Step [0293/0294], Loss:

# Test Data

In [46]:
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# import torch

# from pytorch_transformers import WarmupLinearSchedule, AdamW
# from transformers import get_linear_schedule_with_warmup

# def map_to_torch_float(encoding):
#     encoding = torch.FloatTensor(encoding)
#     encoding.requires_grad_(False)
#     return encoding

# def map_to_torch(encoding):
#     encoding = torch.LongTensor(encoding)
#     encoding.requires_grad_(False)
#     return encoding


class TestTRECDataset(Dataset):
    def __init__(self, raw_data, q_field, d_field, query_max_len=20, max_len=512, is_train=True):
        self.is_train = is_train
        self.max_len = max_len
        self.query_max_len = query_max_len
        
        self.q_field = q_field
        self.d_field = d_field

        self.qLookup = defaultdict()
        self.dLookup = defaultdict()
        self.data_df = self.load_data(raw_data)
        self.len = len(self.data_df)
        
    def load_data(self, data):
        return_data = {'qID': [], 'qText': [], 'dID': [], 'dText': [], 'label': []}
        for data_sample in data:
            qID = data_sample['qID']
            # qText = data_sample['qString']
            qText = '<SEP>'.join( [ e['entity_title'] for e in data_sample['qEntities'] ] )

            if qText == '': continue
            self.qLookup[qID] = qText

            for rel_docs in data_sample['RelevantDocuments']:
                dID = rel_docs['docID']
                # docText = rel_docs['docText']
                docText = '<SEP>'.join( [ e['entity_title'] for e in rel_docs['dEntities'] ] )
                
                if docText == '': continue
                self.dLookup[dID] = docText

                return_data['qID'].append(qID)
                return_data['dID'].append(dID)
                # return_data['qText'].append( qText )
                # return_data['dText'].append( docText )

        # df = pd.DataFrame(return_data, columns=["qID", "qText", "dID", "dText"])
        df = pd.DataFrame(return_data, columns=["qID", "dID"])
        return df
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        index = index % self.len
        inst = self.data_df.loc[index]
        docID = inst['dID']
        qID = inst['qID']
        docText = self.dLookup[docID] # self.dLookup[docID] # inst['dText']
        qText = self.qLookup[qID] # self.qLookup[qID] # inst['qText']
        
        # print(qID)
        # print(qText)
        # print()
        # print(docID)
        # print(docText)

        # =============== prepare text before feed ===============
        q_text_processed = self.q_field.numericalize( self.q_field.pad([ [l for l in self.q_field.preprocess(qText) if l.strip()] ]) )
        d_text_processed = self.d_field.numericalize( self.d_field.pad([ [l for l in self.d_field.preprocess(docText) if l.strip()] ]) )

        return q_text_processed, d_text_processed, qID, docID
    

test_trec_dataset = TestTRECDataset(test_data, Q_TEXT, DOC_TEXT)
print( test_trec_dataset.__len__() )

q, d, qID, dID = test_trec_dataset.__getitem__(1)

print(q[0].shape)
print(d[0].shape)
# print(q[0][0].shape)
# print(out[1][0].shape)
# print(out[2][0].shape)
# print()
# print(out[0][0].shape)

223331
torch.Size([1, 20])
torch.Size([1, 200])


In [47]:
q_str = []
for qq in q[0][0]:
    q_str.append(Q_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

q_str = []
for qq in d[0][0]:
    q_str.append(DOC_TEXT.vocab.itos[qq])
print(' '.join( q_str) )
print()

aftertaste <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

aftertaste food drink perception food drink mouth food drink tin can aftertaste perception food drink neuroscience evolution aftertaste signal transduction taste receptor adrenergic receptor mouth brain food processing insular cortex aftertaste perception <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

In [48]:
test_dataloader = DataLoader(dataset=test_trec_dataset,
                              batch_size=config.training["test_batch_size"],
                              pin_memory=device,
                              num_workers=config.training['num_workers'],
                              shuffle=True)
n_test_batches = len(test_dataloader)
print("Number of test batches : ", n_test_batches)
net.eval()

Number of test batches :  873


SimScorer(
  (q_embedding): Embedding(2943, 300, padding_idx=1)
  (d_embedding): Embedding(171341, 300, padding_idx=1)
  (q_fc1): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
  )
  (d_fc1): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
  )
)

# test

In [49]:
qID_list = []
paraID_list = []
pScore_list = []
t1 = time.time()
for batch_idx, test_batch_data in enumerate(test_dataloader):
    # Converting these to cuda tensors
    q_text, d_text, qID, passageID = test_batch_data
#     print(type(q_text))
#     x = q_text[0]
#     y = q_text[1]
#     print(x.shape)
#     print(y.shape)
#     break
    q_text, d_text = torch.squeeze(q_text[0].to(device)), torch.squeeze(d_text[0]).to(device)

    with torch.no_grad():
        net_output = net({'text_left': q_text, 'text_right': d_text})
        net_output = net_output.detach().cpu().numpy()

        for i in range(len(qID)):
            qID_list.append(qID[i])
            paraID_list.append(passageID[i])
            pScore_list.append(net_output[i])
    elapsed = format_time(time.time() - t1)
    if batch_idx % 50 == 0:
        print('  Batch {:>5,}  of  {:>5,}  :  processed    Elapsed: {:}.'.format(batch_idx,
                                                                                 n_test_batches,
                                                                                 elapsed))

pScore_list = [float(e) for e in pScore_list]
predicted_df = pd.DataFrame({"qID": qID_list,
                             "pID": paraID_list,
                             "pScore": pScore_list}, columns=["qID", "pID", "pScore"])
if not config.cmd_args['mode'] == "experiment":
    predicted_df.to_csv(os.path.join(config.data['results_dir'], "predictions.csv"))
print()

  Batch     0  of    873  :  processed    Elapsed: 0:00:01.
  Batch    50  of    873  :  processed    Elapsed: 0:00:06.
  Batch   100  of    873  :  processed    Elapsed: 0:00:11.
  Batch   150  of    873  :  processed    Elapsed: 0:00:17.
  Batch   200  of    873  :  processed    Elapsed: 0:00:22.
  Batch   250  of    873  :  processed    Elapsed: 0:00:28.
  Batch   300  of    873  :  processed    Elapsed: 0:00:33.
  Batch   350  of    873  :  processed    Elapsed: 0:00:39.
  Batch   400  of    873  :  processed    Elapsed: 0:00:44.
  Batch   450  of    873  :  processed    Elapsed: 0:00:50.
  Batch   500  of    873  :  processed    Elapsed: 0:00:55.
  Batch   550  of    873  :  processed    Elapsed: 0:01:01.
  Batch   600  of    873  :  processed    Elapsed: 0:01:07.
  Batch   650  of    873  :  processed    Elapsed: 0:01:12.
  Batch   700  of    873  :  processed    Elapsed: 0:01:20.
  Batch   750  of    873  :  processed    Elapsed: 0:01:26.
  Batch   800  of    873  :  processed  

In [50]:
# ================================================
#               Reverse Sorting Relevance
# ================================================
predicted_df = predicted_df[['qID', 'pID', 'pScore']]
grouped_pred_df = predicted_df.groupby(["qID"])
num_queries = len(grouped_pred_df)
missing_q_sets = 0
save_ranked_file = os.path.join(config.data['results_dir'], "ranked.test.relevance.txt")
with open(save_ranked_file, 'w') as write_file:
    q_cnt = 1
    for name, row_group in grouped_pred_df:
        rank_cnt = 1

        # ======= SORTING =======
        sorted_row_group = row_group.sort_values(by='pScore', ascending=False, inplace=False)
        # =======================

        if len(sorted_row_group) != 100:
            # print(">>>>>>>>>>> Missing query %s with shape %s" % (name, sorted_row_group.shape))
            # print(">>>>>>>>>>> Missing query with size %s" % sorted_row_group.shape[0])
            missing_q_sets += 1

        for i, row in sorted_row_group.iterrows():
            write_file.write("%s\tQ0\t%s\t%s\t%s\trchan31\n" % \
                             (row["qID"], row["pID"], rank_cnt, row["pScore"]))
            rank_cnt += 1

        print("Finished composing for query number : %s / %s" % (q_cnt, num_queries))
        q_cnt += 1
print()
print("Missing query-doc pairs : ", missing_q_sets)
print("Done train, val, and test !!!")

Finished composing for query number : 1 / 2236
Finished composing for query number : 2 / 2236
Finished composing for query number : 3 / 2236
Finished composing for query number : 4 / 2236
Finished composing for query number : 5 / 2236
Finished composing for query number : 6 / 2236
Finished composing for query number : 7 / 2236
Finished composing for query number : 8 / 2236
Finished composing for query number : 9 / 2236
Finished composing for query number : 10 / 2236
Finished composing for query number : 11 / 2236
Finished composing for query number : 12 / 2236
Finished composing for query number : 13 / 2236
Finished composing for query number : 14 / 2236
Finished composing for query number : 15 / 2236
Finished composing for query number : 16 / 2236
Finished composing for query number : 17 / 2236
Finished composing for query number : 18 / 2236
Finished composing for query number : 19 / 2236
Finished composing for query number : 20 / 2236
Finished composing for query number : 21 / 2236
F

Finished composing for query number : 181 / 2236
Finished composing for query number : 182 / 2236
Finished composing for query number : 183 / 2236
Finished composing for query number : 184 / 2236
Finished composing for query number : 185 / 2236
Finished composing for query number : 186 / 2236
Finished composing for query number : 187 / 2236
Finished composing for query number : 188 / 2236
Finished composing for query number : 189 / 2236
Finished composing for query number : 190 / 2236
Finished composing for query number : 191 / 2236
Finished composing for query number : 192 / 2236
Finished composing for query number : 193 / 2236
Finished composing for query number : 194 / 2236
Finished composing for query number : 195 / 2236
Finished composing for query number : 196 / 2236
Finished composing for query number : 197 / 2236
Finished composing for query number : 198 / 2236
Finished composing for query number : 199 / 2236
Finished composing for query number : 200 / 2236
Finished composing f

Finished composing for query number : 354 / 2236
Finished composing for query number : 355 / 2236
Finished composing for query number : 356 / 2236
Finished composing for query number : 357 / 2236
Finished composing for query number : 358 / 2236
Finished composing for query number : 359 / 2236
Finished composing for query number : 360 / 2236
Finished composing for query number : 361 / 2236
Finished composing for query number : 362 / 2236
Finished composing for query number : 363 / 2236
Finished composing for query number : 364 / 2236
Finished composing for query number : 365 / 2236
Finished composing for query number : 366 / 2236
Finished composing for query number : 367 / 2236
Finished composing for query number : 368 / 2236
Finished composing for query number : 369 / 2236
Finished composing for query number : 370 / 2236
Finished composing for query number : 371 / 2236
Finished composing for query number : 372 / 2236
Finished composing for query number : 373 / 2236
Finished composing f

Finished composing for query number : 530 / 2236
Finished composing for query number : 531 / 2236
Finished composing for query number : 532 / 2236
Finished composing for query number : 533 / 2236
Finished composing for query number : 534 / 2236
Finished composing for query number : 535 / 2236
Finished composing for query number : 536 / 2236
Finished composing for query number : 537 / 2236
Finished composing for query number : 538 / 2236
Finished composing for query number : 539 / 2236
Finished composing for query number : 540 / 2236
Finished composing for query number : 541 / 2236
Finished composing for query number : 542 / 2236
Finished composing for query number : 543 / 2236
Finished composing for query number : 544 / 2236
Finished composing for query number : 545 / 2236
Finished composing for query number : 546 / 2236
Finished composing for query number : 547 / 2236
Finished composing for query number : 548 / 2236
Finished composing for query number : 549 / 2236
Finished composing f

Finished composing for query number : 710 / 2236
Finished composing for query number : 711 / 2236
Finished composing for query number : 712 / 2236
Finished composing for query number : 713 / 2236
Finished composing for query number : 714 / 2236
Finished composing for query number : 715 / 2236
Finished composing for query number : 716 / 2236
Finished composing for query number : 717 / 2236
Finished composing for query number : 718 / 2236
Finished composing for query number : 719 / 2236
Finished composing for query number : 720 / 2236
Finished composing for query number : 721 / 2236
Finished composing for query number : 722 / 2236
Finished composing for query number : 723 / 2236
Finished composing for query number : 724 / 2236
Finished composing for query number : 725 / 2236
Finished composing for query number : 726 / 2236
Finished composing for query number : 727 / 2236
Finished composing for query number : 728 / 2236
Finished composing for query number : 729 / 2236
Finished composing f

...

Finished composing for query number : 1400 / 2236
Finished composing for query number : 1401 / 2236
Finished composing for query number : 1402 / 2236
Finished composing for query number : 1403 / 2236
Finished composing for query number : 1404 / 2236
Finished composing for query number : 1405 / 2236
Finished composing for query number : 1406 / 2236
Finished composing for query number : 1407 / 2236
Finished composing for query number : 1408 / 2236
Finished composing for query number : 1409 / 2236
Finished composing for query number : 1410 / 2236
Finished composing for query number : 1411 / 2236
Finished composing for query number : 1412 / 2236
Finished composing for query number : 1413 / 2236
Finished composing for query number : 1414 / 2236
Finished composing for query number : 1415 / 2236
Finished composing for query number : 1416 / 2236
Finished composing for query number : 1417 / 2236
Finished composing for query number : 1418 / 2236
Finished composing for query number : 1419 / 2236


Finished composing for query number : 1572 / 2236
Finished composing for query number : 1573 / 2236
Finished composing for query number : 1574 / 2236
Finished composing for query number : 1575 / 2236
Finished composing for query number : 1576 / 2236
Finished composing for query number : 1577 / 2236
Finished composing for query number : 1578 / 2236
Finished composing for query number : 1579 / 2236
Finished composing for query number : 1580 / 2236
Finished composing for query number : 1581 / 2236
Finished composing for query number : 1582 / 2236
Finished composing for query number : 1583 / 2236
Finished composing for query number : 1584 / 2236
Finished composing for query number : 1585 / 2236
Finished composing for query number : 1586 / 2236
Finished composing for query number : 1587 / 2236
Finished composing for query number : 1588 / 2236
Finished composing for query number : 1589 / 2236
Finished composing for query number : 1590 / 2236
Finished composing for query number : 1591 / 2236


Finished composing for query number : 1749 / 2236
Finished composing for query number : 1750 / 2236
Finished composing for query number : 1751 / 2236
Finished composing for query number : 1752 / 2236
Finished composing for query number : 1753 / 2236
Finished composing for query number : 1754 / 2236
Finished composing for query number : 1755 / 2236
Finished composing for query number : 1756 / 2236
Finished composing for query number : 1757 / 2236
Finished composing for query number : 1758 / 2236
Finished composing for query number : 1759 / 2236
Finished composing for query number : 1760 / 2236
Finished composing for query number : 1761 / 2236
Finished composing for query number : 1762 / 2236
Finished composing for query number : 1763 / 2236
Finished composing for query number : 1764 / 2236
Finished composing for query number : 1765 / 2236
Finished composing for query number : 1766 / 2236
Finished composing for query number : 1767 / 2236
Finished composing for query number : 1768 / 2236


Finished composing for query number : 1914 / 2236
Finished composing for query number : 1915 / 2236
Finished composing for query number : 1916 / 2236
Finished composing for query number : 1917 / 2236
Finished composing for query number : 1918 / 2236
Finished composing for query number : 1919 / 2236
Finished composing for query number : 1920 / 2236
Finished composing for query number : 1921 / 2236
Finished composing for query number : 1922 / 2236
Finished composing for query number : 1923 / 2236
Finished composing for query number : 1924 / 2236
Finished composing for query number : 1925 / 2236
Finished composing for query number : 1926 / 2236
Finished composing for query number : 1927 / 2236
Finished composing for query number : 1928 / 2236
Finished composing for query number : 1929 / 2236
Finished composing for query number : 1930 / 2236
Finished composing for query number : 1931 / 2236
Finished composing for query number : 1932 / 2236
Finished composing for query number : 1933 / 2236


Finished composing for query number : 2094 / 2236
Finished composing for query number : 2095 / 2236
Finished composing for query number : 2096 / 2236
Finished composing for query number : 2097 / 2236
Finished composing for query number : 2098 / 2236
Finished composing for query number : 2099 / 2236
Finished composing for query number : 2100 / 2236
Finished composing for query number : 2101 / 2236
Finished composing for query number : 2102 / 2236
Finished composing for query number : 2103 / 2236
Finished composing for query number : 2104 / 2236
Finished composing for query number : 2105 / 2236
Finished composing for query number : 2106 / 2236
Finished composing for query number : 2107 / 2236
Finished composing for query number : 2108 / 2236
Finished composing for query number : 2109 / 2236
Finished composing for query number : 2110 / 2236
Finished composing for query number : 2111 / 2236
Finished composing for query number : 2112 / 2236
Finished composing for query number : 2113 / 2236


In [51]:
! ls 

averageOf_cosineSim_cleanedText.ipynb
averageOf_cosineSim_entityOnly.ipynb
config.json
exp1_sim
exp1_sim_trec_results_withTestVocab.txt
exp2_averageOf_cosineSim_entityOnly
exp2_averageOf_cosineSim_entityOnly_trec_results.txt
exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly
exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly-trec_results.txt
extended1FCLayerTrain_averageOf_cosineSim_entityOnly-Copy1.ipynb
extended1FCLayerTrain_averageOf_cosineSim_entityOnly.ipynb


In [41]:
! ls exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly

history.csv		   ranking-pytorch-model
ranked.test.relevance.txt  train-log-epoch.txt


In [42]:
! ../Eval/trec_eval-master/trec_eval ../Eval/qrelsY1-test.V2.0/automatic-test.pages.cbor-hierarchical.qrels exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly/ranked.test.relevance.txt  -m all_trec



runid                 	all	rchan31
num_q                 	all	2236
num_ret               	all	223331
num_rel               	all	6123
num_rel_ret           	all	2349
map                   	all	0.0304
gm_map                	all	0.0018
Rprec                 	all	0.0120
bpref                 	all	0.4689
recip_rank            	all	0.0517
iprec_at_recall_0.00  	all	0.0540
iprec_at_recall_0.10  	all	0.0537
iprec_at_recall_0.20  	all	0.0496
iprec_at_recall_0.30  	all	0.0402
iprec_at_recall_0.40  	all	0.0331
iprec_at_recall_0.50  	all	0.0317
iprec_at_recall_0.60  	all	0.0218
iprec_at_recall_0.70  	all	0.0210
iprec_at_recall_0.80  	all	0.0182
iprec_at_recall_0.90  	all	0.0178
iprec_at_recall_1.00  	all	0.0178
P_5                   	all	0.0125
P_10                  	all	0.0120
P_15                  	all	0.0122
P_20                  	all	0.0114
P_30                  	all	0.0117
P_100                 	all	0.0105
P_200                 	all	0.0053
P_500                 	all	0.0021
P_1000             

In [43]:
! ../Eval/trec_eval-master/trec_eval ../Eval/qrelsY1-test.V2.0/automatic-test.pages.cbor-hierarchical.qrels exp2_averageOf_cosineSim_entityOnly/ranked.test.relevance.txt  -m all_trec > exp3_extended1FCLayerTrain_averageOf_cosineSim_entityOnly-trec_results_moreEpochs.txt


